In [1]:
'''

@Author: Vighnesh Harish Bilgi
@Date: 2022-12-14
@Last Modified by: Vighnesh Harish Bilgi
@Last Modified time: 2022-12-14
@Title : 4 - Pyspark MLlib

'''

'\n\n@Author: Vighnesh Harish Bilgi\n@Date: 2022-12-14\n@Last Modified by: Vighnesh Harish Bilgi\n@Last Modified time: 2022-12-14\n@Title : 4 - Pyspark MLlib\n\n'

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

## Filter operations on Pyspark dataframe

### Creating spark session

In [4]:
spark = SparkSession.builder.appName('Pyspark MLlib').getOrCreate()

In [5]:
spark

### Reading .csv file as training dataset with header and inferSchema as True

In [6]:
training = spark.read.option('header','true').csv('test3.csv', inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [7]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [8]:
training.columns

['Name', 'age', 'Experience', 'Salary']

### Using VectorAssembler() to group specific columns as a feature needed for prediction

In [9]:
featureassembler = VectorAssembler(inputCols=["age","Experience"],outputCol='Independent Features')

In [10]:
output = featureassembler.transform(training)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [11]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [12]:
finalized_data = output.select("Independent Features","Salary")

In [13]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



### Splitting dataframe into training and test dataset and then fitting the Linear Regression Model

#### Using 'age' and 'Experience' (i.e. 'Independent Features') to predict 'Salary'

In [14]:
train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

In [15]:
regressor.intercept

50000.00000004808

In [16]:
regressor.coefficients

DenseVector([-2000.0, 7000.0])

### Call regressor.evaluate() to predict to get output

In [17]:
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [24.0,3.0]| 20000| 23000.00000000573|
|          [30.0,8.0]| 25000|46000.000000025975|
|         [31.0,10.0]| 30000| 58000.00000003782|
+--------------------+------+------------------+



### Error metrics

In [18]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(17333.33333335651, 411333333.3344145)